In [35]:
import requests

In [36]:
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

## Open Geojson

In [37]:
bulgaria_gdf = gpd.read_file("data/bg.json")

In [38]:
bulgaria_gdf["name"] = bulgaria_gdf["name"].str.lower()
bulgaria_gdf = bulgaria_gdf.replace("grad sofiya", "sofia (stolitsa)")

## Get Open Fertility data

In [39]:
r = requests.get(
    "https://www.nsi.bg/en/content/2970/total-fertility-rate-statistical-regions-districts-and-place-residence"
)

In [40]:
soup = BeautifulSoup(r.text, "html.parser")

In [41]:
trs = soup.find("table").find_all("tr")[4:]

In [42]:
fertility_dicts = []
for i, tr in enumerate(trs):
    tds = tr.find_all("td")
    if not len(tds):
        continue
    if tds[1].text == "\xa0":
        continue
    fertility_rate = float(tds[1].text)
    region = tds[0].text
    fertility_dicts.append({"name": region.lower(), "rate": fertility_rate})

In [43]:
fertility_df = pd.DataFrame(fertility_dicts)

## Merge data and export

In [44]:
fertility_rate_gdf = bulgaria_gdf.merge(fertility_df, on="name", how="left")

In [45]:
fertility_rate_gdf = fertility_rate_gdf.to_crs(7803)
fertility_rate_gdf.to_file("data/bulgaria_fertility_rate.gpkg")